In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.16.1'

In [9]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        # open file
        time.sleep(0.03)
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            yield (sample_idx,)

    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_files_in_batches,
            output_signature=tf.TensorSpec(shape=(1,), dtype=tf.int64),
            args=(num_samples,),
        )
        pass

In [10]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range (num_epochs):
        for sample in dataset:
            time.sleep(0.01)

In [11]:
%%timeit
benchmark(FileDataset())

418 ms ± 91.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
benchmark(FileDataset().prefetch(1))

385 ms ± 29.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

375 ms ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
dataset = tf.data.Dataset.range(5)
for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [16]:
dataset = dataset.map(lambda x: x**2)
for d in dataset:
    print(d.numpy())

0
1
4
9
16


In [18]:
dataset = dataset.cache()
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [19]:
def mapped_function(s):
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s
FileDataset().map(mapped_function)

<_MapDataset element_spec=TensorSpec(shape=(1,), dtype=tf.int64, name=None)>

In [21]:
%%timeit
benchmark(FileDataset().map(mapped_function), 5)

1.24 s ± 23.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
benchmark(FileDataset().map(mapped_function).cache(), 5)

705 ms ± 206 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
